# Exercise 5

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

## 1. Data

## 2. Model

### 2.1 Approach

### 2.3 Loss Function

For the error function we follow the Maximum Likelihood paradime.

In binary Logistic Regression we search for $h=h_w$ that maximizes:

$$
\Pi_{i: y_i=1} h(x_i) \Pi_{y_i=-1}(1-h(x_i))
$$

But in Multiclass Logistic Regression we search for $h=h_w$ that maximizes:

$$
\Pi_{i: y_i=0} (h_w(x_i))_c \Pi_{i: y_i=1} (h_w(x_i))_c \ldots \Pi_{i: y_i=C-1} (h_w(x_i))_c\\
= \Pi_{c=0}^{C-1} \Pi_{i: y_i=c} (h_w(x_i))_c
$$

We have:

$$
argmax_w \Pi_{c=0}^{C-1} \Pi_{i: y_i=c} (h_w(x_i))_c\\
= argmax_w \Pi_{c=0}^{C-1} \Pi_{i: y_i=c} (softmax(w^t x_i))_c
$$
We will define:
$$
softmax(w^t x_i) = pred_c\\
$$
Therefore:
$$
argmax_w \Pi_{c=0}^{C-1} \Pi_{i: y_i=c} pred_c\\
= argmax_w \sum_{c=0}^{C-1} \sum_{i: y_i=c} ln[pred_c]\\
= argmin_w -\frac{1}{N} \sum_{c=0}^{C-1} \sum_{i: y_i=c} ln[pred_c]
$$

### 2.2 Implementation

In [2]:
def sigmoid(z):
        return 1 / (1 + np.exp(-z))

In [3]:
class LogisticRegression:

    def fit(self, X, y, lr=1e-1, epochs=10):
        
        # Initialize random weights
        self.weights = np.random.standard_normal(X.shape[1])

        # Start training our model
        for e in range(1, epochs + 1):
            print(f"--------------- epoch {e}/{epochs} ---------------")
            
            # Calculate gradient
            gradient = np.zeros_like(self.weights)
            for xi, yi in zip(X, y):
                gradient += sigmoid(-yi * self.weights @ xi) * (-yi) * xi
            
            # Update weights
            self.weights -= lr * gradient

            print(f"  train score: {self.score(X, y)}")

        return self

    def predict_proba(self, X):
        # Use sigmoid to estimate the probability of X being labeled as positive
        return sigmoid(X @ self.weights)

    def predict(self, X, threshold=0.5):
        # Predict the labels according to the threshold
        return 2 * (self.predict_proba(X) >= threshold) - 1

    def score(self, X, y):
        # Calculating the accuracy score of our predictions
        return (self.predict(X) == y).sum() / len(y)

## 3. Experiments

## 4. Evaluation